<a href="https://colab.research.google.com/github/jacaceres1/Modelado/blob/Proyects/%5BNRC_8903%5DJoseC%C3%A1ceres_Lab1Unidad2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import glob #permite buscar la ruta
import pandas as pd #pd es el apodo que recibe para us funcion
import xml.etree.ElementTree as ET #permite construir docuimentos xml
from datetime import datetime #clase para manipular fechas y horas
from zipfile import ZipFile #extraer la libreria para descomprimir
import requests

In [8]:
#mediante la funcion para descargar en segundo plano
def getData():
  remote_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip'
  local_file = 'datasource.zip'
  data = requests.get(remote_url)
  with open(local_file, 'wb') as f:
    f.write(data.content)
  with ZipFile(local_file, 'r') as zipObj:
    zipObj.extractall('dealership_data')


In [2]:
#establecer los archivos de destino
tmpfile = "dealership_temp.tmp"
logfile = "dealership_logfile.txt"
targetfile = "dealership_transformed_data.csv"

In [3]:
#funcion csv 
def extract_from_csv(file_to_process):
  dataframe = pd.read_csv(file_to_process)
  return dataframe

#funcion json
def extract_from_json(file_to_process):
  dataframe = pd.read_json(file_to_process,lines=True)
  return dataframe
#funcion xml
def extract_from_xml(file_to_process):
  dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price','fuel'])
  tree = ET.parse(file_to_process)
  root = tree.getroot()
  for person in root:
    car_model = person.find("car_model").text
    year_of_manufacture = int(person.find("year_of_manufacture").text)
    price = float(person.find("price").text)
    fuel = person.find("fuel").text
    dataframe = dataframe.append({"car_model":car_model,"year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel},ignore_index=True)
  return dataframe

def extract():
  extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
  #for csv files
  for csvfile in glob.glob("dealership_data/*.csv"):
    extracted_data = extracted_data.append(extract_from_csv(csvfile),ignore_index=True)
  #for json files
  for jsonfile in glob.glob("dealership_data/*.json"):
    extracted_data = extracted_data.append(extract_from_json(jsonfile),ignore_index=True)
  #for xml files
  for xmlfile in glob.glob("dealership_data/*.xml"):
    extracted_data = extracted_data.append(extract_from_xml(xmlfile),ignore_index=True)
  return extracted_data

In [4]:
def transform(data):
  data['price'] = round(data.price, 2)
  return data

In [5]:
def load(targetfile,data_to_load):
  data_to_load.to_csv(targetfile)

In [6]:
def log(message):
  timestamp_format = '%H:%M:%S-%h-%d-%Y'
  #Hour-Minute-Second-MonthName-Day-Year
  now = datetime.now() # get current timestamp
  timestamp = now.strftime(timestamp_format)
  with open("dealership_logfile.txt","a") as f: 
    f.write(timestamp + ',' + message +'n')

In [10]:
#Main para ejecutar la descarga
if __name__ == '__main__':
  #declarar todas las funciones qu se van a ejecutar
  getData()

  #extraer los datos descargados
  log("Extract phase Started")
  extracted_data = extract()
  log("Extract phase Ended")

  #transformar los datos obtenidos
  log("Transform phase Started")
  transformed_data = transform(extracted_data)
  log("Transform phase Ended")

  #load 
  log("Load phase Started")
  load(targetfile,transformed_data)
  log("Load phase Ended")

  #finalizar el etl correspondiente
  log("ETL Job Ended")

In [44]:
print("Datos extraidos")
print(extracted_data)


Datos extraidos
        car_model year_of_manufacture     price    fuel
0        alto 800                2017   4253.73  Petrol
1            ciaz                2015  10223.88  Diesel
2            ciaz                2015  11194.03  Petrol
3          ertiga                2015   9104.48  Petrol
4           dzire                2009   3358.21  Petrol
..            ...                 ...       ...     ...
85        etios g                2015   5895.52  Petrol
86  corolla altis                2013   8208.96  Petrol
87        corolla                2004   2238.81  Petrol
88  corolla altis                2010   7835.82  Petrol
89       fortuner                2012  21641.79  Diesel

[90 rows x 4 columns]


In [45]:
print("Datos Extraidos")
print(transformed_data)

Datos Extraidos
        car_model year_of_manufacture     price    fuel
0        alto 800                2017   4253.73  Petrol
1            ciaz                2015  10223.88  Diesel
2            ciaz                2015  11194.03  Petrol
3          ertiga                2015   9104.48  Petrol
4           dzire                2009   3358.21  Petrol
..            ...                 ...       ...     ...
85        etios g                2015   5895.52  Petrol
86  corolla altis                2013   8208.96  Petrol
87        corolla                2004   2238.81  Petrol
88  corolla altis                2010   7835.82  Petrol
89       fortuner                2012  21641.79  Diesel

[90 rows x 4 columns]
